In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import time 
import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

In [2]:
import talib as ta

ModuleNotFoundError: No module named 'talib'

In [ ]:
df1=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXXFUTURE.xlsx","UX1 Index")
df2=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXXFUTURE.xlsx","UX2 Index")

In [ ]:
data1=df1.copy()
data2=df2.copy()
#Set the date index
data1.set_index('Dates',drop=True,inplace=True)
data2.set_index('Dates',drop=True,inplace=True)
#Localize the HK time zone
data1.index=pd.to_datetime(data1.index).tz_localize('Hongkong') #UTC-5
data2.index=pd.to_datetime(data2.index).tz_localize('Hongkong')
#Conver the HK time zone 
data1.index=pd.to_datetime(data1.index).tz_convert('US/Eastern') #UTC-5
data2.index=pd.to_datetime(data2.index).tz_convert('US/Eastern')


In [ ]:
UX1=data1.between_time('09:30','16:00')
UX2=data2.between_time('09:30','16:00')

In [ ]:
UX1['Volatility']=np.log(UX1['High'])-np.log(UX1['Low'])
UX2['Volatility']=np.log(UX2['High'])-np.log(UX2['Low'])
UX1['TIME']=UX1.index.time
UX2['TIME']=UX2.index.time

In [ ]:
UX12=pd.concat([UX1,UX2],axis=1, join='inner')
UX12.columns=['Open','Close','High','Low','Value','Volume','Number Ticks','Volatility','Time','Open2','Close2','High2','Low2','Value2','Volume2','Number Ticks2','Volatility2','Time2']
UX12['Spread']=(UX12['Close2']-UX12['Close'])

### Including Extended Hours

In [ ]:
UX1extended=data1.between_time('15:55','17:00')
UX2extended=data2.between_time('15:55','17:00')
UX1extended['Volatility']=np.log(UX1extended['High'])-np.log(UX1extended['Low'])
UX2extended['Volatility']=np.log(UX2extended['High'])-np.log(UX2extended['Low'])
UX1extended['TIME']=UX1extended.index.time
UX2extended['TIME']=UX2extended.index.time
UX1extended['DAY']=UX1extended.index.dayofweek #Monday=0 Sunday=6 for seasonality grouping
UX2extended['DAY']=UX2extended.index.dayofweek
UX12extended=pd.concat([UX1extended,UX2extended],axis=1, join='inner')
UX12extended.columns=['Open','Close','High','Low','Value','Volume','Number Ticks','Volatility','Time','Day','Open2','Close2','High2','Low2','Value2','Volume2','Number Ticks2','Volatility2','Time2','Day2']
UX12extended['Spread']=(UX12extended['Close2']-UX12extended['Close'])

In [ ]:
UX12['Date']=UX12.index.date

In [ ]:
def add_week_of_month(df):
    df['week_in_month'] = pd.to_numeric(df.index.day/7)
    df['week_in_month'] = df['week_in_month'].apply(lambda x: math.ceil(x))
    return df
UX12extended['Date']=UX12extended.index.date
UX12extended=add_week_of_month(UX12extended)

### Bid Ask Spread

In [ ]:
df1bid=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXBIDASK.xlsx","UX1BID")


In [ ]:
df1ask=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXBIDASK.xlsx","UX1ASK")


In [ ]:
df2bid=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXBIDASK.xlsx","UX2BID")


In [ ]:
df2ask=pd.read_excel("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\VIXBIDASK.xlsx","UX2ASK")

In [ ]:
data1bid=df1bid.copy()
data1ask=df1ask.copy()
data2bid=df2bid.copy()
data2ask=df2ask.copy()

In [ ]:
#Set the date index
data1bid.set_index('Dates',drop=True,inplace=True)
data1ask.set_index('Dates',drop=True,inplace=True)
data2bid.set_index('Dates',drop=True,inplace=True)
data2ask.set_index('Dates',drop=True,inplace=True)
#Localize the HK time zone
data1bid.index=pd.to_datetime(data1bid.index).tz_localize('Hongkong') #UTC-5
data1ask.index=pd.to_datetime(data1ask.index).tz_localize('Hongkong')
data2bid.index=pd.to_datetime(data2bid.index).tz_localize('Hongkong') #UTC-5
data2ask.index=pd.to_datetime(data2ask.index).tz_localize('Hongkong')
#Conver the HK time zone 
data1bid.index=pd.to_datetime(data1bid.index).tz_convert('US/Eastern') #UTC-5
data1ask.index=pd.to_datetime(data1ask.index).tz_convert('US/Eastern')
data2bid.index=pd.to_datetime(data2bid.index).tz_convert('US/Eastern') #UTC-5
data2ask.index=pd.to_datetime(data2ask.index).tz_convert('US/Eastern')

In [ ]:
UX1extended=data1.between_time('15:55','17:00')
UX2extended=data2.between_time('15:55','17:00')
UX1extended['Volatility']=np.log(UX1extended['High'])-np.log(UX1extended['Low'])
UX2extended['Volatility']=np.log(UX2extended['High'])-np.log(UX2extended['Low'])
UX12extended=pd.concat([UX1extended,UX2extended],axis=1, join='inner')
UX12extended.columns=['Open','Close','High','Low','Value','Volume','Number Ticks','Volatility','Time','Day','Open2','Close2','High2','Low2','Value2','Volume2','Number Ticks2','Volatility2','Time2','Day2']
UX12extended['Spread']=(UX12extended['Close2']-UX12extended['Close'])

In [ ]:
UX1extendedbid=data1bid.between_time('15:55','17:00')
UX1extendedask=data1ask.between_time('15:55','17:00')
UX2extendedbid=data2bid.between_time('15:55','17:00')
UX2extendedask=data2ask.between_time('15:55','17:00')
UX12extendedbidask=pd.concat([UX1extendedbid,UX1extendedask,UX2extendedbid,UX2extendedask],axis=1, join='inner')
UX12extendedbidask.columns=['OpenB1','CloseB1','HighB1','LowB1','ValueB1','VolumeB1','Number TicksB1','OpenA1','CloseA1','HighA1','LowA1','ValueA1','VolumeA1','Number TicksA1','OpenB2','CloseB2','HighB2','LowB2','ValueB2','VolumeB2','Number TicksB2','OpenA2','CloseA2','HighA2','LowA2','ValueA2','VolumeA2','Number TicksA2']

In [ ]:
UX12extendedbidask.at_time('15:55')[['CloseB1','CloseA1','CloseB2','CloseA2']]

### Normal + Extended

In [ ]:
#to CSV
UX1.to_csv("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\UX1_clean.csv")
UX2.to_csv("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\UX2_clean.csv")
UX12.to_csv("C:\\Users\\raymond.widjaja\\Desktop\\VIX\\UX12_clean.csv")

# Analysis

In [ ]:
VOL1=UX1.groupby('TIME').mean()
VOL2=UX2.groupby('TIME').mean()


In [ ]:
#VOL1['Volatility'].plot(figsize=(16,8))
VOL1['Volume'].plot(figsize=(16,8))
#VOL1[['Close','Open']].plot(figsize=(16,8))
plt.title('First Month Vix Futures Price Average across time')
plt.axvline('15:56:00', color='r')
#plt.axvline('16:00:00', color='g')

In [ ]:
#VOL2['Volatility'].plot(figsize=(16,8))
#VOL2['Volume'].plot(figsize=(16,8))
VOL2[['Close','Open']].plot(figsize=(16,8))
#VOL1['Number Ticks'].plot(figsize=(16,8))
plt.title('Second Month Vix Futures Price Average across time')
plt.axvline('15:56:00', color='r')
#plt.axvline('15:40:00', color='g')

In [ ]:
UX1.groupby('TIME').describe()

In [ ]:
UX12['Close'].plot(figsize=(16,8),label='UX1')
UX12['Close2'].plot(figsize=(16,8),label='UX2')
plt.legend()
plt.show()

In [ ]:
VOL12=UX12.groupby('Time').mean()

In [ ]:
VOL12['Spread'].plot(figsize=(16,8))
plt.title('Spread')
plt.axvline('15:00:00', color='r')

# Analysis (Contango: Specific)

In [ ]:
std

In [ ]:
std=UX12.describe()[['Spread']].T['std'][0]

In [ ]:
UX12Contango=UX12[UX12['Spread']>0.1*std]
UX12Back=UX12[UX12['Spread']<-0.1*std]
UX12Neutral=UX12[(UX12['Spread']<0.1*std)&(UX12['Spread']>-0.1*std)]

In [ ]:
#UX12['Spread'].plot.hist(bins=30,alpha=0.8)
sns.distplot(UX12['Spread'], hist=True, kde=True, 
             bins=int(180/5), color = 'blue', 
             hist_kws={'edgecolor':'red'},
             kde_kws={'linewidth': 2})
plt.title('Density Plot and Histogram of the Spread')


### Contango

In [ ]:
UX12Contango
Cont=UX12Contango.groupby('Time').mean()
Cont[['Volume']].plot(figsize=(16,8),label='UX1')
plt.title('Contango VIX 1st Month Volume')
plt.axvline('15:56:00', color='r')
plt.show()

In [ ]:
Cont[['Volume2']].plot(figsize=(16,8),label='UX2')
plt.title('Contango VIX 2nd Month Volume')
plt.axvline('15:56:00', color='r')
plt.show()

In [ ]:
Cont['Volatility'].plot(figsize=(16,8))
plt.title("Contago VIX 1st month Volatility")
plt.axvline('15:00:00', color='r')

In [ ]:
Cont['Volatility2'].plot(figsize=(16,8))
plt.title("Contago VIX 2nd month Volatility")
plt.axvline('15:00:00', color='r')

In [ ]:
Cont[['Close','Open']].plot(figsize=(16,8))
plt.title('Contango 2nd Month Vix Futures Price Average across time')
plt.axvline('15:00:00', color='r')
#plt.axvline('15:40:00', color='g')

### Backwardation

In [ ]:
UX12Back
Back=UX12Back.groupby('Time').mean()
Back[['Volume']].plot(figsize=(16,8),label='UX1')
plt.title('Backwardation VIX 1st Month Volume')
plt.axvline('15:56:00', color='r')
plt.axvline('9:47:00', color='g')
plt.axvline('9:53:00', color='g')
#plt.axvline('10:28:00', color='g')
#plt.axvline('10:40:00', color='g')
plt.show()

In [ ]:
Back[['Volume2']].plot(figsize=(16,8),label='UX2')
plt.title('Backwardation VIX 2nd Month Volume')
plt.axvline('15:56:00', color='r')
plt.axvline('10:57:00', color='g')
plt.axvline('11:07:00', color='g')
plt.show()

In [ ]:
Back['Volatility'].plot(figsize=(16,8)) #/volatility2
plt.title("Backwardation VIX 1st month Volatility")
plt.axvline('15:00:00', color='r')

In [ ]:
Back[['Close','Open']].plot(figsize=(16,8))
plt.title('Backwardation 1st Month Vix Futures Price Average across time')
plt.axvline('15:00:00', color='r')
#plt.axvline('15:40:00', color='g')

### Neutral

In [ ]:
UX12Neutral
Neutral=UX12Neutral.groupby('Time').mean()
Neutral[['Volume']].plot(figsize=(16,8),label='UX1')
plt.title('Neutral VIX 1st Month Volume')
plt.axvline('15:56:00', color='r')
plt.show()

In [ ]:
Neutral[['Volume2']].plot(figsize=(16,8),label='UX2')
plt.title('Neutral VIX 2nd Month Volume')
#plt.axvline('15:56:00', color='r')
plt.show()

In [ ]:
Neutral['Volatility'].plot(figsize=(16,8))
plt.title("Neutral VIX 1st month Volatility")
plt.axvline('15:00:00', color='r')
plt.axvline('13:58:00', color='g')
plt.axvline('14:03:00', color='g')

In [ ]:
Neutral['Volatility2'].plot(figsize=(16,8))
plt.title("Neutral VIX 2nd month Volatility")
plt.axvline('15:00:00', color='r')
plt.axvline('13:58:00', color='g')
plt.axvline('14:03:00', color='g')

In [ ]:
Neutral[['Close2','Open2']].plot(figsize=(16,8))
plt.title('Neutral 2nd Month Vix Futures Price Average across time')
plt.axvline('15:00:00', color='r')
plt.axvline('15:30:00', color='g')

# After Extended Hour

In [ ]:
VOL12extended=UX12extended.groupby('Time').mean()

In [ ]:
(UX12extended['Spread']<0).sum()


In [ ]:
VOL12extended['Volume2'].plot(figsize=(16,8))
#VOL12extended[['Close','Open']].plot(figsize=(16,8))
plt.title('2nd Month Vix Futures Volume Average across time')
plt.axvline('16:14:00', color='r')
#plt.axvline('16:00:00', color='g')

In [ ]:
VOL12extended[['Close2','Open2']].plot(figsize=(16,8))
plt.title('2nd Month Vix Futures Price Average across time')
plt.axvline('16:30:00', color='r')
plt.axvline('16:50:00', color='g')

In [ ]:
VOL12extended['Volatility'].plot(figsize=(16,8))
plt.title('1st Month Vix Futures Volatility Average across time')
plt.axvline('16:30:00', color='r')

In [ ]:
VOL12extended['Value2'].plot(figsize=(16,8))
plt.title('2nd Month Vix Futures Value Average across time')
plt.axvline('16:30:00', color='r')

In [ ]:
VOL12extended.head()

In [ ]:
def fm_summary(p):#Summary Statistics Function
    t=pd.Series((v for v in p ))
    s = t.describe().T
    s['Up Mean']=p[p>0].mean()
    s['Down Mean']=p[p<0].mean()
    s['Up Median']=p[p>0].median()
    s['Down Median']=p[p<0].median()
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    s['Up']=t[t>0].count()
    s['Down']=t[t<0].count()
    s['breakeven']=t[t==0].count()
    s['Up(%)']=t[t>0].count()/s['count']
    s['Down(%)']=t[t<0].count()/s['count']
    return s[['count','mean','Up','Down','breakeven','Up Mean','Down Mean','Up Median','Down Median','Up(%)','Down(%)']].T


# Contango, Backwardation and Neutral State

In [ ]:
stde=UX12extended.describe()[['Spread']].T['std'][0]
UX12extendedContango=UX12extended[UX12extended['Spread']>0.1*std]
UX12extendedBack=UX12extended[UX12extended['Spread']<-0.1*std]
UX12extendedNeutral=UX12extended[(UX12extended['Spread']<0.1*std)&(UX12extended['Spread']>-0.1*std)]

In [ ]:
UX12extended.describe()[['Spread']]

In [ ]:
contbidask=pd.concat([UX12extendedContango['Spread'],UX12extendedbidask[['CloseB1','CloseA1','CloseB2','CloseA2']]],axis=1,join='inner')
contbidask.columns=['Spread','Bid1','Ask1','Bid2','Ask2']
contbidask

# Contango [MAIN]

In [ ]:
UX12extendedContango
ContExt=UX12extendedContango.groupby('Time').mean()
ContExt[['Spread']].plot(figsize=(16,8),label='UX1')
plt.title('Contango VIX 1nd Month Price')
plt.axvline('15:55:00', color='b')
plt.axvline('16:00:00', color='r')
plt.axvline('16:14:00', color='y')
plt.axvline('16:05:00', color='g')
#plt.show()

In [ ]:
a=UX12extendedContango.at_time('15:55').set_index('Date')['Spread']
b=UX12extendedContango.at_time('16:00').set_index('Date')['Spread']
c=UX12extendedContango.at_time('16:05').set_index('Date')['Spread']
d=UX12extendedContango.at_time('16:14').set_index('Date')['Spread']
f=UX12Contango.at_time('15:00').set_index('Date')['ATRSpread']
g=UX12Contango.at_time('15:55').set_index('Date')['ATRSpread']
contangostat=pd.concat([a,b,c,d,f,g],axis=1,join='inner')
contangostat.columns=['State1','State2','State3','Closing','ATR15:00','ATR15:55']
contangostat['Singal']=(contangostat['ATR15:55']/contangostat['ATR15:00'])-1
contangostat['Long at 15:55']=contangostat['Closing']-contangostat['State1']
contangostat['Long at 16:00']=contangostat['Closing']-contangostat['State2']
contangostat['Long at 16:05']=contangostat['Closing']-contangostat['State3']


### ATR Strategy Development

In [ ]:
contangostat.to_excel('C:\\Users\\raymond.widjaja\\Desktop\\VIX\\statistics.xlsx')

In [ ]:
contangostatt

In [ ]:
#Change in ATR in the last hour is greater than 0.2
contangostatt=contangostat[contangostat['Singal']>0.2]

In [ ]:
contangostatt2=contangostat[((contangostat['Singal']<-0.2))] #change in the ATR in the last hour is least than 0.2

In [ ]:
model=sm.OLS(contangostat['Long at 15:55'],contangostat['Singal'])


### Long

In [ ]:
contSUM=pd.concat([fm_summary(contangostatt['Long at 15:55']),fm_summary(contangostatt['Long at 16:00']),fm_summary(contangostatt['Long at 16:05'])],axis=1)
contSUM.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUM=contSUM.round(3)
contSUM.style.set_caption("CONTANGO")

### Short

In [ ]:
contSUM2=pd.concat([fm_summary(contangostatt2['Long at 15:55']),fm_summary(contangostatt2['Long at 16:00']),fm_summary(contangostatt2['Long at 16:05'])],axis=1)
contSUM2.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUM2=contSUM2.round(3)
contSUM2.style.set_caption("CONTANGO")

In [ ]:
sns.distplot(contangostat['Long at 15:55'], hist=True, kde=True, 
             bins=int(180/5), color = 'blue', 
             hist_kws={'edgecolor':'red'},
             kde_kws={'linewidth': 2})
plt.title('Density Plot and Histogram of the Long')
plt.axvline(0)


In [ ]:
sns.scatterplot(contangostat['Singal'],contangostat['Long at 15:55'])
plt.axhline(0)
plt.axvline(0.2,color='red')

### With bid ask spread

In [ ]:
contbidask['bidasktotalspread']=((contbidask['Ask1']-contbidask['Bid1'])+(contbidask['Ask2']-contbidask['Bid2']))/2
contbidask['bidasktotalspread']

In [ ]:
contbidask[contbidask['bidasktotalspread']<0].head(5)

In [ ]:
aS=contbidask.at_time('15:55')
bS=contbidask.at_time('16:00')
cS=contbidask.at_time('16:05')
dS=contbidask.at_time('16:14')
eS=contbidask.at_time('16:12')
aS=aS['bidasktotalspread']
bS=bS['bidasktotalspread']
cS=cS['bidasktotalspread']
dS=dS['bidasktotalspread']
eS=eS['bidasktotalspread']
aS.index=aS.index.date
bS.index=bS.index.date
cS.index=cS.index.date
dS.index=dS.index.date
eS.index=eS.index.date

In [ ]:
contangostatt2
fm_summary(contangostatt2['Long at 15:55'])

### ATR strategy development

### Long

In [ ]:
contangostatS=pd.concat([contangostatt,aS,bS,cS,dS,eS],axis=1,join='inner') #Slight change contangstatt / contangostat
list2=contangostat.columns.tolist()
list2.append('bs1')
list2.append('bs2')
list2.append('bs3')
list2.append('bs4')
list2.append('bs5')
contangostatS.columns=list2

In [ ]:
#contangostatS.head(20)

In [ ]:
contangostatS['Long at 15:55 w/ spread']=contangostatS['Long at 15:55']-(contangostatS['bs1']+contangostatS['bs4'])
contangostatS['Long at 16:00 w/ spread']=contangostatS['Long at 16:00']-(contangostatS['bs2']+contangostatS['bs4'])
contangostatS['Long at 16:05 w/ spread']=contangostatS['Long at 16:05']-(contangostatS['bs3']+contangostatS['bs4'])


In [ ]:
contangostatS['Long at 15:55 pct change']=contangostatS['Long at 15:55 w/ spread']/contangostatS['State1']


## Backtest Summary

In [ ]:
def fm_summaryS(p,p2):#Summary Statistics Function
    t=pd.Series((v for v in p ))
    t2=pd.Series((w for w in p2))
    s = t.describe().T
    t3=pd.concat([t[t>0],t2],axis=1,join='inner') #Up
    t3[1]
    t4=pd.concat([t[t<0],t2],axis=1,join='inner') #Down
    
    s['Up Mean']=t3[1].mean()
    s['Down Mean']=t4[1].mean()
    s['Up Median']=t3[1].median()
    s['Down Median']=t4[1].median()
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    s['Up']=t[t>0].count()
    s['Down']=t[t<0].count()
    s['breakeven']=t[t==0].count()
    s['Up(%)']=t[t>0].count()/s['count']
    s['Down(%)']=t[t<0].count()/s['count']
    s['Expected Value']=(s['Up(%)']*s['Up Mean'])+(s['Down Mean']*s['Down(%)'])
    return s[['count','mean','Up','Down','breakeven','Up Mean','Down Mean','Up Median','Down Median','Up(%)','Down(%)','Expected Value']].T


In [ ]:
pd.concat([contangostatt2,aS,bS,cS,dS,eS],axis=1,join='inner')

In [ ]:
def losedate(p1):
    ldate=p1[p1['Long at 15:55']<0].index.tolist()
    return ldate
    

In [ ]:
contSUMbidask=pd.concat([fm_summaryS(contangostatS['Long at 15:55'],contangostatS['Long at 15:55 w/ spread']),fm_summaryS(contangostatS['Long at 16:00'],contangostatS['Long at 16:00 w/ spread']),fm_summaryS(contangostatS['Long at 16:05'],contangostatS['Long at 16:05 w/ spread'])],axis=1)
contSUMbidask.columns=['Long 15:55 w/S','Long 16:00 w/S','Long 16:05 w/S']
contSUMbidask=contSUMbidask.round(4)
contSUMbidask.style.set_caption("CONTANGO with average spread [ Assume we just take LONG position when change in ATR>0.2 ]")

In [ ]:
import pyfolio as pf

In [ ]:
stratret=contangostatS['Long at 15:55 pct change'].copy()
stratret.index=pd.to_datetime(stratret.index).tz_localize('US/Eastern')

In [ ]:
contangostatS['cumret15:55']=(1+contangostatS['Long at 15:55 pct change']).cumprod()

In [ ]:
contangostatS

# =====================

In [ ]:
atr
contSUMbidask=pd.concat([fm_summaryS(contangostatS['Long at 15:55'],contangostatS['Long at 15:55 w/ spread']),fm_summaryS(contangostatS['Long at 16:00'],contangostatS['Long at 16:00 w/ spread']),fm_summaryS(contangostatS['Long at 16:05'],contangostatS['Long at 16:05 w/ spread'])],axis=1)
contSUMbidask.columns=['Long 15:55 w/S','Long 16:00 w/S','Long 16:05 w/S']
contSUMbidask=contSUMbidask.round(4)
contSUMbidask.style.set_caption("CONTANGO with average spread [ Assume we just take LONG position ]")

In [ ]:

UX12Contango['ATR']=ta.ATR((UX12Contango['High2']-UX12Contango['High']),(UX12Contango['Low2']-UX12Contango['Low']),(UX12Contango['Close2']-UX12Contango['Close']),timeperiod=14)
UX12Contango['Spread'].plot(figsize=(16,8),label='UX1')
plt.legend()
plt.title('Contango')
plt.show()

In [ ]:
#ATR spread calculuation 
UX12Contango['Close'].plot(figsize=(16,8),label='UX1')
UX12Contango['Close2'].plot(figsize=(16,8),label='UX2')
UX12Contango['ATR']=ta.ATR(UX12Contango['High'],UX12Contango['Low'],UX12Contango['Close'],timeperiod=14)
UX12Contango['ATR2']=ta.ATR(UX12Contango['High2'],UX12Contango['Low2'],UX12Contango['Close2'],timeperiod=14)
UX12Contango['SpreadH']=UX12Contango['High2']-UX12Contango['Low']
UX12Contango['SpreadL']=UX12Contango['Low2']-UX12Contango['High']
UX12Contango['ATRSpread']=ta.ATR(UX12Contango['SpreadH'],UX12Contango['SpreadL'],UX12Contango['Spread'],timeperiod=14)
plt.legend()
plt.axvline('15:55', color='g')
plt.show()


### Change datetime index to date

In [ ]:
testCT=UX12Contango.copy()

In [ ]:
testCT.index=testCT.index.date
testCT
testCT['Close'].iloc[:20000].plot(figsize=(16,8),label='UX1')
testCT['Close2'].iloc[:20000].plot(figsize=(16,8),label='UX2')

plt.axvline(lodate[0], color='r')
plt.axvline(lodate[1], color='r')
plt.axvline(lodate[2], color='r')
plt.axvline(lodate[3], color='r')
plt.axvline(lodate[4], color='r')
plt.axvline(lodate[5], color='r')
plt.axvline(lodate[6], color='r')
plt.axvline(lodate[7], color='r')
plt.axvline(lodate[8], color='r')
plt.axvline(lodate[9], color='r')
plt.axvline(lodate[10], color='r')
plt.axvline(lodate[11], color='r')
plt.axvline(lodate[12], color='r')
plt.axvline(lodate[13], color='r')
#for i in lodate:
    #plt.axvline(i, color='r')
    
plt.legend()

In [ ]:
testCT
testCT['Close'].iloc[20000:].plot(figsize=(16,8),label='UX1')
testCT['Close2'].iloc[20000:].plot(figsize=(16,8),label='UX2')

plt.axvline(lodate[14], color='r')
plt.axvline(lodate[15], color='r')
plt.axvline(lodate[16], color='r')
plt.axvline(lodate[17], color='r')
plt.axvline(lodate[18], color='r')
plt.axvline(lodate[19], color='r')
plt.axvline(lodate[20], color='r')
plt.axvline(lodate[21], color='r')
plt.axvline(lodate[22], color='r')
plt.axvline(lodate[23], color='r')
plt.axvline(lodate[24], color='r')
plt.axvline(lodate[25], color='r')
plt.axvline(lodate[26], color='r')
plt.axvline(lodate[27], color='r')
#for i in lodate:
    #plt.axvline(i, color='r')
    
plt.legend()

In [ ]:
contangostat

In [ ]:
lodate=losedate(contangostatS)


In [ ]:
losses=pd.DataFrame(pd.DatetimeIndex(lodate).dayofweek.tolist(),columns=['Day'])
losses['Down']=1
losses.groupby('Day').count()

In [ ]:
testCT['ATR'].iloc[:20000].plot(figsize=(16,8),label='ATR UX1')
testCT['ATR2'].iloc[:20000].plot(figsize=(16,8),label='ATR UX2')
plt.legend()
#for i in lodate:
    #plt.axvline(i, color='r')
plt.axvline(lodate[0], color='r')
plt.axvline(lodate[1], color='r')
plt.axvline(lodate[2], color='r')
plt.axvline(lodate[3], color='r')
plt.axvline(lodate[4], color='r')
plt.axvline(lodate[5], color='r')
plt.axvline(lodate[6], color='r')
plt.axvline(lodate[7], color='r')
plt.axvline(lodate[8], color='r')
plt.axvline(lodate[9], color='r')
plt.axvline(lodate[10], color='r')
plt.axvline(lodate[11], color='r')
plt.axvline(lodate[12], color='r')
plt.axvline(lodate[13], color='r')

In [ ]:
testCT['ATR'].iloc[20000:].plot(figsize=(16,8),label='ATR UX1')
testCT['ATR2'].iloc[20000:].plot(figsize=(16,8),label='ATR UX2')
plt.legend()
#for i in lodate:
    #plt.axvline(i, color='r')
plt.axvline(lodate[14], color='r')
plt.axvline(lodate[15], color='r')
plt.axvline(lodate[16], color='r')
plt.axvline(lodate[17], color='r')
plt.axvline(lodate[18], color='r')
plt.axvline(lodate[19], color='r')
plt.axvline(lodate[20], color='r')
plt.axvline(lodate[21], color='r')
plt.axvline(lodate[22], color='r')
plt.axvline(lodate[23], color='r')
plt.axvline(lodate[24], color='r')
plt.axvline(lodate[25], color='r')
plt.axvline(lodate[26], color='r')
plt.axvline(lodate[27], color='r')


## ATR ANALYSIS

In [ ]:
UX12Contango['ATR'].loc[datetime.datetime(2020,5,15,9,30):datetime.datetime(2020,5,15,16,0)].plot(figsize=(16,8),label='ATR UX1')
UX12Contango['ATR2'].loc[datetime.datetime(2020,5,15,9,30):datetime.datetime(2020,5,15,16,0)].plot(figsize=(16,8),label='ATR UX2')
plt.title('ATR 15 May')
plt.axhline(UX12Contango['ATR'].loc[datetime.datetime(2020,5,15,9,45)]/2)
plt.legend()

## ATR SPREAD

In [ ]:
m=9
d=3
beg=datetime.datetime(2020,m,d,9,30)
end=datetime.datetime(2020,m,d,16,0)
aZ=datetime.datetime(2020,m,d,19,55)#CHANGE TO 20:55 AFTER DAY IGHT SAVING HOUR
aT=datetime.datetime(2020,m,d,19,0)
beg1=datetime.datetime(2020,m,d,9,45)
begext=datetime.datetime(2020,m,d,15,55)
endext=datetime.datetime(2020,m,d,16,14)

In [ ]:
UX12Contango['ATRSpread'].loc[beg:end].plot(figsize=(16,8),label='ATR Spread') 
plt.title('ATR Spread 8 June')
#plt.axhline(UX12Contango['ATRSpread'].loc[beg1]/2)
UX12Contango['ATRSpread'].rolling(150).mean().loc[beg:end].plot(figsize=(16,8),label='ATRSpread MA150') 
plt.legend()
plt.axvline(aZ,color='r')
plt.axvline(aT,color='r')
plt.axhline(0.11)
plt.axhline(0.15)

In [ ]:
UX12Contango['ATRSpread'].loc[beg:end].tail(7)

In [ ]:
0.297

In [ ]:
plt.subplot(1,2,1)
UX12Contango['Spread'].loc[beg:end].plot(figsize=(16,8))
plt.title('Spread')
UX12Contango['Spread'].rolling(70).mean().loc[beg:end].plot(figsize=(16,8))
ag=datetime.datetime(2020,m,d,19,55)
plt.axvline(ag,color='r')
plt.subplot(1,2, 2)
UX12extendedContango['Spread'].loc[begext:endext].plot(figsize=(16,8))
plt.title('Spread in ext')
aT=datetime.datetime(2020,m,d,20,14)  # 21:`13AFTER DLIGHT SAVING
plt.axvline(aT,color='r')

In [ ]:
UX12Contango['Spread'].rolling(90).mean().loc[beg:end].tail(7)

In [ ]:
UX12Contango

In [ ]:
UX12Contango['ATR'].loc[datetime.datetime(2020,6,2,9,30):datetime.datetime(2020,6,2,16,0)]

In [ ]:
print(ta.get_function_groups())

In [ ]:
UX12Contango

In [ ]:
contangostat

## ======================================

In [ ]:
contangostat['Thres']=contangostat['State1'].rolling(5).mean()+ 1.5*contangostat['State1'].rolling(3).std()
contangostat['Thres2']=contangostat['State2'].rolling(5).mean()+ 1.5*contangostat['State2'].rolling(3).std()
contangostat['Thres3']=contangostat['State3'].rolling(5).mean()+ 1.5*contangostat['State3'].rolling(3).std()
contangostat['Thres3']=contangostat['State3'].rolling(5).mean()+ 1.5*contangostat['State3'].rolling(3).std()
contangostat['prev']=contangostat['State1'].shift(1)
contangostat['prev2']=contangostat['State2'].shift(1)
contangostat['prev3']=contangostat['State2'].shift(1)

### Long contango if today price is lower than yesterday price

In [ ]:
da=fm_summary(contangostat[contangostat['State1']<1.5*contangostat['prev']]['Long at 15:55'])
db=fm_summary(contangostat[contangostat['State2']<1.5*contangostat['prev2']]['Long at 16:00'])
dc=fm_summary(contangostat[contangostat['State3']<1.5*contangostat['prev3']]['Long at 16:05'])
contSUMst=pd.concat([da,db,dc],axis=1)
contSUMst.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUMst=contSUMst.round(2)
contSUMst.style.set_caption("CONTANGO Normal [Today price < 1.5 *yesterday price]")

### Long contango without extreme Contango

In [ ]:
da2=fm_summary(contangostat[contangostat['State1']<contangostat['Thres']]['Long at 15:55'])
db2=fm_summary(contangostat[contangostat['State2']<contangostat['Thres2']]['Long at 16:00'])
dc2=fm_summary(contangostat[contangostat['State3']<contangostat['Thres3']]['Long at 16:05'])
contSUMst2=pd.concat([da2,db2,dc2],axis=1)
contSUMst2.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUMst2=contSUMst2.round(2)
contSUMst2.style.set_caption("CONTANGO without extreme contango")

In [ ]:
contangostat.head(25)

## Backwardation

In [ ]:
UX12extendedBack
BackExt=UX12extendedBack.groupby('Time').mean()
BackExt[['Close2']].plot(figsize=(16,8),label='UX1')
plt.title('Backwardation VIX 2nd Month Price')
plt.axvline('16:14:00', color='r')
plt.axvline('16:30:00', color='y')
plt.axvline('16:59:00', color='y')
plt.show()

In [ ]:
aa=UX12extendedBack.at_time('15:55').set_index('Date')['Spread']
bb=UX12extendedBack.at_time('16:00').set_index('Date')['Spread']
cc=UX12extendedBack.at_time('16:05').set_index('Date')['Spread']
dd=UX12extendedBack.at_time('16:14').set_index('Date')['Spread']
contangostat2=pd.concat([aa,bb,cc,dd],axis=1,join='inner')
contangostat2.columns=['State1','State2','State3','Closing']
contangostat2['Long at 15:55']=contangostat2['Closing']-contangostat2['State1']
contangostat2['Long at 16:00']=contangostat2['Closing']-contangostat2['State2']
contangostat2['Long at 16:05']=contangostat2['Closing']-contangostat2['State3']

In [ ]:
contSUM2=pd.concat([fm_summary(contangostat2['Long at 15:55']),fm_summary(contangostat2['Long at 16:00']),fm_summary(contangostat2['Long at 16:05'])],axis=1)
contSUM2.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUM2=contSUM2.astype('float64').round(2)
contSUM2.style.set_caption("Backwardation")

## Neutral

In [ ]:
UX12extendedNeutral
NeutralExt=UX12extendedNeutral.groupby('Time').mean()
NeutralExt[['Close2']].plot(figsize=(16,8),label='UX1')
plt.title('Neutral VIX 2nd Month Price')
plt.axvline('16:14:00', color='r')
plt.axvline('16:30:00', color='y')
plt.axvline('16:59:00', color='y')
plt.show()

In [ ]:
aaa=UX12extendedNeutral.at_time('15:55').set_index('Date')['Spread']
bbb=UX12extendedNeutral.at_time('16:00').set_index('Date')['Spread']
ccc=UX12extendedNeutral.at_time('16:05').set_index('Date')['Spread']
ddd=UX12extendedNeutral.at_time('16:14').set_index('Date')['Spread']
contangostat3=pd.concat([aaa,bbb,ccc,ddd],axis=1,join='inner')
contangostat3.columns=['State1','State2','State3','Closing']
contangostat3['Long at 15:55']=contangostat3['Closing']-contangostat3['State1']
contangostat3['Long at 16:00']=contangostat3['Closing']-contangostat3['State2']
contangostat3['Long at 16:05']=contangostat3['Closing']-contangostat3['State3']

In [ ]:
contSUM3=pd.concat([fm_summary(contangostat3['Long at 15:55']),fm_summary(contangostat3['Long at 16:00']),fm_summary(contangostat3['Long at 16:05'])],axis=1)
contSUM3.columns=['Long 15:55','Long 16:00','Long 16:05']
contSUM3=contSUM3.astype('float64').round(2)
contSUM3.style.set_caption("Neutral")

In [ ]:
UX12extendedNeutral.head()

### Dissecting the Factor

In [ ]:
#Extreme Contago 
t=pd.Series((v for v in UX12extendedContango.between_time('15:55','16:15')['Spread']))
s = t.describe()
s



## State 1


In [ ]:
contangostat['State1'].rolling(5).mean()

In [ ]:
contangostat.head(20)

## State 2

## State 3

In [ ]:
strat=UX12extendedContango.between_time('15:55','16:15')
strat

# Seasonality Analysis

In [ ]:
UX1s=data1.between_time('09:30','17:00')
UX2s=data2.between_time('09:30','17:00')
UX1s['Volatility']=np.log(UX1s['High'])-np.log(UX1s['Low'])
UX2s['Volatility']=np.log(UX2s['High'])-np.log(UX2s['Low'])
UX1s['TIME']=UX1s.index.time
UX2s['TIME']=UX2s.index.time
UX1s['DAY']=UX1s.index.dayofweek #Monday=0 Sunday=6 for seasonality grouping
UX2s['DAY']=UX2s.index.dayofweek
UX12s=pd.concat([UX1s,UX2s],axis=1, join='inner')
UX12s.columns=['Open','Close','High','Low','Value','Volume','Number Ticks','Volatility','Time','Day','Open2','Close2','High2','Low2','Value2','Volume2','Number Ticks2','Volatility2','Time2','Day2']
UX12s['Spread']=(UX12s['Close2']-UX12s['Close'])

In [ ]:
season=UX12s.groupby(['Day','Time']).mean()
season

In [ ]:
mon=season.loc[0,:]
tue=season.loc[1,:]
wed=season.loc[2,:]
thurs=season.loc[3,:]
fri=season.loc[4,:]

In [ ]:
mon['Volume'].plot(figsize=(16,8),label='mon')
tue['Volume'].plot(figsize=(16,8),label='tue')
wed['Volume'].plot(figsize=(16,8),label='wed')
thurs['Volume'].plot(figsize=(16,8),label='thurs')
fri['Volume'].plot(figsize=(16,8),label='fri')
plt.title('1st Month Vix Futures Volume Average across different day')
plt.legend()
plt.show()
#plt.axvline('16:15:00', color='r')

In [ ]:
mon.iloc[390:,:]['Volume'].plot(figsize=(16,8),label='mon')
tue.iloc[390:,:]['Volume'].plot(figsize=(16,8),label='tue')
wed.iloc[390:,:]['Volume'].plot(figsize=(16,8),label='wed')
thurs.iloc[390:,:]['Volume'].plot(figsize=(16,8),label='thurs')
fri.iloc[390:,:]['Volume'].plot(figsize=(16,8),label='fri')
plt.title('1st Month Vix Futures Volume Average across different day')
plt.legend()
plt.show()
#plt.axvline('16:15:00', color='r')

In [ ]:
mon['Close2'].plot(figsize=(16,8),label='mon')
tue['Close2'].plot(figsize=(16,8),label='tue')
wed['Close2'].plot(figsize=(16,8),label='wed')
thurs['Close2'].plot(figsize=(16,8),label='thurs')
fri['Close2'].plot(figsize=(16,8),label='fri')
plt.title('1st Month Vix Futures Price Average across different day')
plt.legend()
plt.show()
#plt.axvline('16:15:00', color='r')

In [ ]:
mon.iloc[390:,:]['Close2'].plot(figsize=(16,8),label='mon')
tue.iloc[390:,:]['Close2'].plot(figsize=(16,8),label='tue')
wed.iloc[390:,:]['Close2'].plot(figsize=(16,8),label='wed')
thurs.iloc[390:,:]['Close2'].plot(figsize=(16,8),label='thurs')
fri.iloc[390:,:]['Close2'].plot(figsize=(16,8),label='fri')
plt.title('1st Month Vix Futures Price Average across different day')
plt.legend()
plt.show()
#plt.axvline('16:15:00', color='r')

### ETF 